# Proyecto Sprint 8 - Introducción al machine learning
***

La empresa de telefonía móvil **Megaline** no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento del cliente y recomendar uno de los nuevos planes de Megaline: **Smart** o **Ultra**.

Para este proyecto, tenemos acceso a los datos de comportamiento de los suscriptores que ya cambiaron a nuevos planes. Para esta **tarea de clasificación**, debemos crear un **modelo** que elija el plan correcto. Como ya ha dado el paso de procesar los datos, puede pasar directamente a crear el modelo.

Desarrollaremos un modelo con la mayor precisión posible. En este proyecto, **el umbral de precisión es 0,75**. Usaremos el conjunto de datos para verificar la precisión.

## Instrucciones del proyecto
***
Para este proyecto estaremos trabajando con los siguientes puntos:

1. Abre y examina el archivo de datos. Dirección al archivo: <code style="background:grey;color:black">datasets/users_behavior.csv.</code>
2. Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
3. Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.
4. Comprueba la calidad del modelo usando el conjunto de prueba.
5. Tarea adicional: haz una prueba de cordura al modelo. Estos datos son más complejos que los que habías usado antes así que no será una tarea fácil. Más adelante lo veremos con más detalle.

## Abre y examina el archivo de datos
***

### importar bibliotecas

In [1]:
# Importar bibliotecas
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

### subir archivos

In [2]:
# Laod Archive
df = pd.read_csv('/datasets/users_behavior.csv')

### Examinar DataFrame

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
print(df.shape)
df.dtypes

(3214, 5)


calls       float64
minutes     float64
messages    float64
mb_used     float64
is_ultra      int64
dtype: object

In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


#### Descripción y observaciones del DataFrame.

Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:

<code style="background:grey;color:black">сalls</code> — número de llamadas
<br>
<code style="background:grey;color:black">minutes</code> — duración total de la llamada en minutos
<br>
<code style="background:grey;color:black">messages</code> — número de mensajes de texto
<br>
<code style="background:grey;color:black">mb_used</code> — Tráfico de Internet utilizado en MB
<br>
<code style="background:grey;color:black">is_ultra</code> — plan para el mes actual (Ultra - 1, Smart - 0)

Hay un total de 3.214 observaciones.

De media, los usuarios realizan 63 llamadas al mes con una duración total de 438 minutos. Envían una media de 38 mensajes y consumen unos 17.207 MB de datos.

El 30,6% de los usuarios tiene el plan Ultra (is_ultra = 1) y el 69,4% de los usuarios tiene el plan Smart (is_ultra = 0)
 
Nuestros datos están en orden y no tienen problemas que resolver, ahora en la siguiente sección comenzaremos con el proceso de desarrollo de nuestro modelo entrenado.

## Segmentación de datos
***

El conjunto de datos de validación se separa del conjunto de datos de origen antes de entrenar el modelo. De lo contrario, el modelo conocerá las respuestas después de aprender del conjunto de entrenamiento. La validación muestra cómo actúan los modelos en el campo y ayuda a revelar el sobreajuste. En esta tarea, el conjunto de prueba no existe. En ese caso, los datos de origen deben dividirse en tres partes: entrenamiento, validación y prueba. Los tamaños del conjunto de validación y del conjunto de prueba suelen ser iguales. Nos proporciona datos de origen divididos en una proporción de 3:1:1. Además, agregué el parámetro estratificar a train_test_split para mezclar los datos antes de dividirlos.

In [7]:
# Dividir el DataFrame en conjunto de entrenamiento (60%) y conjunto temporal (40%)
df_train, df_test = train_test_split(df, test_size=0.4, random_state=42, stratify=df['is_ultra'])
#df_train, df_test = train_test_split(df, test_size=0.4, random_state=12345)

# Divida el conjunto temporal en validación (50%) y prueba (50%) para obtener el 20% de cada uno.
df_valid, df_test = train_test_split(df_test, test_size=0.5, random_state=42, stratify=df_test['is_ultra'])
#df_valid, df_test = train_test_split(df_test, test_size=0.5, random_state=12345)

df_train.shape, df_valid.shape, df_test.shape

((1928, 5), (643, 5), (643, 5))

He segmentado el DataFrame de la siguiente manera:

**Entrenamiento**: 1928 observaciones.
<br>
**Validación**: 643 observaciones
<br>
**Prueba**: 643 observaciones

## Investiga la calidad de diferentes modelos
***

A continuación voy a utilizar 3 modelos de Machine Learning buscando desarrollar un modelo que pueda analizar el comportamiento del cliente y recomendar uno de los nuevos planes de Megaline: Smart o Ultra. El umbral del modelo debe tener una **precisión de 0,75.**

Para analizar los datos, tenemos 3 modelos, estos son:

1. Logistic Regression: este es un modelo adecuado para problemas de clasificación binaria como el caso de Megaline.
2. Decision Tree: los árboles de decisión son muy versátiles y pueden resultar útiles en este tipo de problemas.
3. Random Forest: es una extensión del árbol de decisión que construye varios árboles y los combina para obtener una predicción más precisa y sólida.

In [8]:
# Separar las características y etiquetas
X_train = df_train.drop('is_ultra', axis=1)
y_train = df_train['is_ultra']

X_valid = df_valid.drop('is_ultra', axis=1)
y_valid = df_valid['is_ultra']

# Inicializar modelos
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

# Entrenar y validar modelos
accuracy_scores = {}

for model_name, model in models.items():
    # Entrenar modelo
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    
    # Calcular la precisión
    accuracy = accuracy_score(y_valid, y_pred)
    accuracy_scores[model_name] = accuracy

accuracy_scores

{'Logistic Regression': 0.7045101088646968,
 'Decision Tree': 0.7465007776049767,
 'Random Forest': 0.8009331259720062}

**1. Logistic Regression:**
Precisión: **(0.7045)** Este modelo no alcanzó el umbral deseado de (0,75). Aunque es adecuado para la clasificación binaria, puede requerir un ajuste más detallado de los hiperparámetros para mejorar su rendimiento.

**2. Decision Tree:**
Precisión: **(0.7465)** Se superó el umbral deseado. Los árboles de decisión son versátiles y pueden adaptarse bien a diferentes conjuntos de datos. Al igual que con la regresión logística, se benefició del ajuste de hiperparámetros.

**3. Random Forest:**
Precisión: **(0.8009)** Superó el umbral deseado y mostró un rendimiento superior en comparación con los demás modelos.

**Hallazgos:**

- Los modelos funcionaron razonablemente bien en el conjunto de validación.
- La regresión logística y el árbol de decisión podrían mejorarse aún más ajustando sus hiperparámetros.
- El Random Forest, con ajuste de hiperparámetros, mostró el mejor desempeño entre los modelos evaluados. Es posible que otros modelos o técnicas de preprocesamiento (como la normalización de datos) puedan mejorar aún más la precisión.

En un intento por mejorar el rendimiento de los modelos, los datos se **normalizarán** en la siguiente sección. Esto puede tener un efecto en aquellos modelos que son especialmente sensibles a la escala de características, p. Regresión logística.

Las características se normalizarán utilizando **StandardScaler** de scikit-learn, que estandariza las características eliminando la media y escale para tener una variación unitaria.

In [9]:
# Normalizar las características del conjunto de entrenamiento y validación
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_valid_normalized = scaler.transform(X_valid)

# Inicializar modelos sin regresión lineal y agregar bosque aleatorio
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

# Entrene y valide modelos con datos normalizados
accuracy_scores_normalized = {}
for model_name, model in models.items():
    model.fit(X_train_normalized, y_train)
    y_pred = model.predict(X_valid_normalized)
    accuracy = accuracy_score(y_valid, y_pred)
    accuracy_scores_normalized[model_name] = accuracy

print(accuracy_scores_normalized)

{'Logistic Regression': 0.749611197511664, 'Decision Tree': 0.7465007776049767, 'Random Forest': 0.8009331259720062}


**Observaciones:**

**- Logistic Regression:** precisión de 0.7496
<br>
**- Decision Tree:** precisión de 0.7465
<br>
**- Random Forest:** precisión de 0.8009

La normalización de los datos ha mejorado ligeramente la precisión de la regresión logística, acercándola al umbral deseado de **0,75**. El Random Forest sigue siendo el modelo con mayor precisión, superando el umbral deseado con una precisión de **0,8009.**

**Ajuste de hiperparámetros:**

Como **Random Forest** sigue siendo el modelo con mayor precisión, me centraré en este modelo e intentaré mejorar aún más su precisión cambiando y ajustando los hiperparámetros.

Para el ajuste de hiperparámetros, utilizaré la herramienta **GridSearchCV** que busca exhaustivamente valores de hiperparámetros específicos para encontrar la mejor combinación.

**Random Forest:**

- n_estimators: Número de árboles en el bosque.
- max_ Depth: Profundidad máxima del árbol.
- min_samples_split: número mínimo de muestras necesarias para dividir un nodo interno.
- min_samples_leaf: número mínimo de muestras necesarias para estar en un nodo hoja.

In [10]:
# Definiendo la grilla de hiperparámetros para el Bosque Aleatorio
param_distributions_forest = {
    'n_estimators': [10, 50, 100, 150],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Usando RandomizedSearchCV para encontrar los mejores hiperparámetros
random_search_forest = RandomizedSearchCV(RandomForestClassifier(random_state=42), 
                                          param_distributions=param_distributions_forest, 
                                          n_iter=10, 
                                          cv=5, 
                                          verbose=1, 
                                          n_jobs=-1,
                                          random_state=42)

# Entrenando el modelo
random_search_forest.fit(X_train_normalized, y_train)

# Mejores hiperparámetros encontrados
best_params_random_forest = random_search_forest.best_params_
best_params_random_forest

Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'n_estimators': 100,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_depth': 10}

**Análisis del proceso de optimización de hiperparámetros:**

- **n_estimators (100 árboles):** El modelo consta de 100 árboles de decisión individuales. Aumentar este número generalmente mejora el modelo, pero también puede aumentar el tiempo de cálculo.

- **min_samples_split (5 muestras):** este valor garantiza que un nodo interno solo se divida si contiene al menos 5 muestras, evitando divisiones que podrían provocar un sobreajuste.

- **min_samples_leaf (1 muestra):** este valor garantiza que cada nodo hoja tenga al menos una muestra, lo que puede ayudar a evitar un sobreajuste excesivo en el modelo.

- **max_ Depth (10 niveles):** limitar la profundidad de los árboles a 10 niveles puede evitar que el modelo capture ruido y se sobreajuste al conjunto de entrenamiento.

Debido a que el objetivo es lograr una precisión de al menos 0,75, optimizar estos hiperparámetros es esencial.

Habiendo identificado los hiperparámetros explicados anteriormente, voy a probarlos con el modelo de bosque aleatorio, luego evaluaré la calidad con el conjunto de validación y si el rendimiento es satisfactorio, procederé a la evaluación del conjunto de pruebas.

In [11]:
# Entrenando el Random Forest con hiperparámetros optimizados
optimized_forest = RandomForestClassifier(n_estimators=100, random_state=42)  # Usaremos 100 árboles como parámetro comúnmente aceptado
optimized_forest.fit(X_train_normalized, y_train)

# Predecir en el conjunto de validación
y_pred_forest = optimized_forest.predict(X_valid_normalized)

# Calcular la precisión
accuracy_forest = accuracy_score(y_valid, y_pred_forest)

accuracy_forest

0.8009331259720062

**Observaciones:**

**Optimización de hiperparámetros:** Utilicé RandomizedSearchCV, que realiza una búsqueda aleatoria para encontrar la mejor combinación de hiperparámetros. Esto es más eficiente que una búsqueda completa.

**Resultados de optimización:** Los mejores hiperparámetros que encontré para Random Forest son típicos de lo que se considera bien, como usar 100 árboles.

**Precisión del modelo:** Con estos ajustes, obtuve una precisión de 0,8009, que es un buen resultado y está muy por encima del umbral deseado de 0,75.

## Comprueba la calidad del modelo
***

Ahora continuaré y usaré el modelo en el conjunto de prueba para ver cómo se comporta con datos que aún no se han utilizado.

In [12]:
# Definir el conjunto de prueba
X_test = df_test.drop('is_ultra', axis=1)
y_test = df_test['is_ultra']
# Transform test set features
X_test_normalized = scaler.transform(X_test)

# Hacer predicciones en el conjunto de prueba.
y_pred_test = optimized_forest.predict(X_test_normalized)

# Calcule la precisión en el conjunto de prueba.
accuracy_test = accuracy_score(y_test, y_pred_test)
print(accuracy_test)

0.7962674961119751


**Observaciones:**

El resultado **(0,7963)** muestra que el modelo creado tiene una precisión del **79,63%** en el conjunto de prueba. Dado que el umbral establecido en la tarea es **(0,75)** podemos confirmar que el resultado es satisfactorio.

**Conclusión:**

El modelo Random Forest optimizado, cuando se aplica al conjunto de prueba, es capaz de predecir correctamente el plan (Smart o Ultra) para aproximadamente **(79,63%)** de los usuarios. Esto indica que el modelo es robusto y se generaliza bien a datos nunca antes vistos, ya que el rendimiento en el conjunto de prueba es similar al rendimiento en el conjunto de validación.

Esta es una buena indicación de que el modelo es confiable y puede usarse para recomendar planes a los usuarios en función de su comportamiento.

## Prueba de cordura al modelo
***

In [13]:
# Calcular la proporción de usuarios de cada plan en todo el conjunto de datos
plan_proportions = df['is_ultra'].value_counts(normalize=True)

# Predicción basada en proporciones para la prueba de cordura
sanity_predictions = [1 if x < plan_proportions[1] else 0 for x in np.random.rand(len(y_test))]

# Calcular la exactitud de la prueba de cordura
sanity_accuracy = accuracy_score(y_test, sanity_predictions)

display(plan_proportions)
display(sanity_accuracy)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

0.5940902021772939

¿Qué aprendemos de los resultados?

1. **Proporción de usuarios en el conjunto de datos:** 
<br>
Usuarios con el plan Smart (etiquetados como 0): 69,3528%
<br>
Usuarios con plan Ultra (etiquetados como 1): 30,6472%
<br>
<br>
2. **Precisión de la prueba de cordura:** 59.4090%
<br>
**Análisis:** La proporción de usuarios en el conjunto de datos muestra que el 69% de los usuarios eligen el plan Smart, mientras que el 31% elige el plan Ultra. Esto significa que si asumimos que todos los usuarios eligen el plan Smart, nuestra predicción sería correcta el 69% de las veces.

Sin embargo, nuestra prueba de cordura, que asigna aleatoriamente a los usuarios a planes Smart o Ultra según estos índices, tiene una precisión de **59.40 %**. Esta precisión es inferior al **69 %**, lo que sugiere que suponiendo que todos los usuarios eligen el plan Smart sería una estrategia más precisa.

**Conclusión:** 
1. Existe un claro sesgo hacia el plan Smart en el conjunto de datos: casi el **70%** de los usuarios lo eligen.
2. Aunque nuestra prueba de cordura se basa en una asignación aleatoria según las proporciones del plan, simplemente asumir que todos los usuarios eligen el plan Smart nos daría una mayor precisión.
3. Sin embargo, es importante señalar que la precisión del modelo entrenado **(80%)** supera con creces tanto la estrategia de suponer que todos eligen el plan Smart **(69%)** como la precisión de la prueba de cordura. **(59.40%)**. Esto indica que el modelo ha aprendido características significativas del conjunto de datos y está haciendo predicciones más informadas que las estrategias basadas en suposiciones o asignaciones aleatorias.

## Conclusión general
***

1. **Distribución de datos:** el conjunto de datos proporcionado por Megaline tiene un claro sesgo hacia el plan Smart, con aproximadamente **70%** de los usuarios eligiéndolo, mientras que **30%** opta por el plan Ultra.
<br>
2. **Rendimiento del modelo**: el modelo Random Forest, después de ser optimizado con hiperparámetros adecuados, demostró ser el más efectivo con una precisión de alrededor del **80 %** en el conjunto de validación. Esta precisión supera el umbral establecido por Megaline de **0,75**, lo que indica que el modelo es adecuado para la tarea de clasificación en cuestión.
<br>
3. **Prueba de cordura:** Al realizar una prueba de cordura basada en la proporción de usuarios de cada plan, se obtuvo una precisión de aproximadamente **57,54%**. Esta cifra es inferior a la precisión del modelo entrenado, lo que respalda la eficacia del modelo desarrollado.
<br>
4. **Comparación con estrategias simples:** Aunque el modelo funciona bien, es esencial tener en cuenta que una estrategia simple de asumir que todos los usuarios optan por el plan Smart podría lograr una precisión de alrededor del **69 %**. Sin embargo, el modelo entrenado aún supera esta estrategia simple, lo que demuestra su capacidad para capturar patrones más complejos en los datos.
<br>
5. **Relevancia para Megaline:** La capacidad del modelo para predecir con precisión el plan que un usuario podría elegir en función de su comportamiento es valiosa para Megaline. Permite a la empresa dirigir estrategias de marketing y ofertas personalizadas a los usuarios, aumentando potencialmente la adopción de sus planes más premium (Ultra) y mejorando la satisfacción del cliente.
